# Task C - Voronoi
## Divide city into N=10 cells


# Description

1. Select the initial set of 10 cell seed points. For this, you can use several criteria, such as being far away from frequent accident roads, being close to public transport, being evenly spread, etc. (explain your choice in the report).
2. Visualise the cells yield by your selection of seed points in a Voronoi diagram. What kind of Voronoi diagram (edge planar, node network, or edge points network) is most useful for this problem, and why?
3. Find 2 or 3 cells for which you can find at least one path (or more, if possible) that is (a) exactly 42 Km long, and (b) finishes at the same point where it starts. Visualise both the cells and the found paths.
4. Try to extend the previous step to all cells. Can you find at least one such a path for every cell?
5. If for steps 3-4 there were cells with no such path, what different options could you consider to increase the number of cells that include such paths? (Hint: think about the number and location of seed points; the size of the area under consideration; etc.) Choose one of such options, rep

## TODO
- Fix seeder points -> evenly spread
- what voronoi diagram??  edge planar, node network, or edge points network
- Find a path in each cell of 42 km
- 42km path in a circle

In [1]:
import networkx as nx
import osmnx as ox
import random as rnd
import matplotlib.pyplot as plt
from taskC.voronoi_tutorial_helpers import nodes_nearest_seed, get_seed_color, map_node_color_from_seed, map_edge_color_from_node
# from taskC.seeder import get_seeds_kMeans
import numpy as np
import folium
import geopandas as gpd
from shapely.geometry import Point
from sklearn.cluster import KMeans
import pickle


In [2]:
N_SEEDS = 10
# query_place_graph = ox.graph_from_place('Leeds, United Kingdom', network_type='all')

# with open('my_graph.pickle', 'wb') as f:
#     pickle.dump(query_place_graph, f)

# Loading
with open('my_graph.pickle', 'rb') as f:
    query_place_graph = pickle.load(f)

What type of Voronoi diagram 

In [ ]:
# nx.check_planarity

In [ ]:
def get_seeds_kMeans(graph, num_seeds: int):
    # Extract node coordinates
    coords = [(node, data['x'], data['y']) for node, data in graph.nodes(data=True)]
    gdf_nodes = gpd.GeoDataFrame(coords, columns=['node_id', 'x', 'y'])

    # Apply K-means clustering
    kmeans = KMeans(n_clusters=num_seeds)
    kmeans.fit(gdf_nodes[['x', 'y']])
    print(type(kmeans.cluster_centers_[0]))

    # Extract cluster centroids as seeds
    unaligned_seeds = [Point(x, y) for x, y in kmeans.cluster_centers_]
    return unaligned_seeds

def get_nearest_edge(G, point):
    # Get all edges in the graph
    gdf = ox.graph_to_gdfs(G, nodes=False, fill_edge_geometry=True)
    graph_edges = gdf["geometry"].values.tolist()

    # Compute the Euclidean distance from the coordinates to each edge
    edges_with_distances = [
        (graph_edge, Point(point).distance(graph_edge))
        for graph_edge in graph_edges
    ]

    # Sort edges based on distance
    edges_with_distances.sort(key=lambda x: x[1])

    # Return the closest edge (geometry)
    return edges_with_distances[0]

N_SEEDS = 10
unaligned_seeds = get_seeds_kMeans(query_place_graph, N_SEEDS)
# seeds = [get_nearest_edge(query_place_graph, seed) for seed in unaligned_seeds]
# seeds

In [ ]:
def get_seeds_kMeans(graph, num_seeds: int):
    # 2. Extract node coordinates
    coords = [(node, data['x'], data['y']) for node, data in graph.nodes(data=True)]
    gdf_nodes = gpd.GeoDataFrame(coords, columns=['node_id', 'x', 'y'])

    # 3. Apply K-means clustering
    kmeans = KMeans(n_clusters=num_seeds)
    kmeans.fit(gdf_nodes[['x', 'y']])

    # 4. Extract cluster centroids as seeds
    unaligned_seeds = [Point(x, y) for x, y in kmeans.cluster_centers_]
    return unaligned_seeds

def get_nearest_edge(G, point):
    # Get all edges in the graph
    gdf = ox.graph_to_gdfs(G, nodes=False, fill_edge_geometry=True)
    graph_edges = gdf["geometry"].values.tolist()

    # Compute the Euclidean distance from the coordinates to each edge
    edges_with_distances = [
        (graph_edge, Point(tuple(reversed(point))).distance(graph_edge))
        for graph_edge in graph_edges
    ]

    # Sort edges based on distance
    edges_with_distances.sort(key=lambda x: x[1])

    # Return the closest edge (geometry)
    return edges_with_distances[0]


In [ ]:
unaligned_seeds = get_seeds_kMeans(query_place_graph, N_SEEDS)
unaligned_seeds
# print(unaligned_seeds)
# print(type(unaligned_seeds[0]))
# 
# seeds = [get_nearest_edge(query_place_graph, seed) for seed in unaligned_seeds]
# seeds

In [ ]:
all_nodes = list(query_place_graph.nodes)

# TODO: choose seeds carefully, with respect to idk yet 
seeds = rnd.choices(all_nodes, k=10)
cells = nx.voronoi_cells(query_place_graph, seeds, weight='length')
cells

In [ ]:
black_color = (0.0, 0.0, 0.0, 1.0)

node_seed_dict = nodes_nearest_seed(query_place_graph, seeds, cells)
seed_colors = get_seed_color(seeds, black_color)
node_color_dict = map_node_color_from_seed(query_place_graph, node_seed_dict, seed_colors)
edge_colors = map_edge_color_from_node(query_place_graph, node_seed_dict, node_color_dict, black_color)

In [ ]:
node_colors = ['r' if node in seeds else 'w' for node in all_nodes]
ox.plot.plot_graph(query_place_graph, edge_color=edge_colors, node_color=node_colors, bgcolor ='k', save=True, filepath='nvd.png', node_size=1, figsize=(15, 15))

- Find a path in each cell of 42 km
- 42km path in a circle

In [3]:
all_nodes = list(query_place_graph.nodes)

# TODO: choose seeds carefully, with respect to idk yet 
seeds = rnd.choices(all_nodes, k=10)
cells = nx.voronoi_cells(query_place_graph, seeds, weight='length')

In [5]:
from taskC.distance_metrices import CustomDFS
import multiprocessing as mp


cell = cells[seeds[1]]

custom_dfs = CustomDFS(query_place_graph, cell)
custom_dfs.dfs()

TypeError: 'NoneType' object is not subscriptable

In [4]:
for i in range(10):
    print(f"iteration: {i}\n", len(cells[seeds[i]]))

iteration: 0
 8414
iteration: 1
 955
iteration: 2
 4260
iteration: 3
 10971
iteration: 4
 7090
iteration: 5
 13232
iteration: 6
 24412
iteration: 7
 5905
iteration: 8
 4830
iteration: 9
 18485
